In [ ]:
#!pip install mysqlclient
#!pip install bellmanford
#!pip install networkx
#!pip install ccxt

In [ ]:
from OrderbookAnalyser import OrderbookAnalyser

In [ ]:
runLocal = True

if runLocal == False:
    host="orderbook-2.cyifbgm0zwt0.eu-west-2.rds.amazonaws.com"
    user="admin"
    passwd="123Qwe123Qwe"
    db="orderbook"
    port=3306
else:
    host="127.0.0.1"
    user="admin"
    passwd="admin"
    db="orderbook"
    port=33306

orderbookAnalyser = OrderbookAnalyser(host,user,passwd,db,port)

exchangeList = ['coinfloor','kraken','bitfinex','bittrex','gdax','bitstamp','coinbase','poloniex']
vol_BTC = 0.15
df=orderbookAnalyser.runSim(exchangeList=exchangeList,limit=None,vol_BTC=vol_BTC)

In [ ]:
df.head()

In [ ]:
df['profit_perc'].hist(bins=100)

In [ ]:
df.to_csv("./results/arbitrage_Vol=%2.3fBTC.csv"%vol_BTC,index=False)

In [ ]:
df[['exchanges_involved', 'profit_perc']].groupby(['exchanges_involved']).agg(['min','max','mean', 'count']).sort_values([('profit_perc', 'mean')], ascending=False)